In [ ]:
import requests
import json
import pprint
import pandas as pd
import time

#bot leblanc
#api_key = ' '
#초코

# grandmaster = 'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
# challenger = 'https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
# master = 'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key

# r = requests.get(grandmaster)# 그마데이터 호출
# 데이터 호출


def call_rank_data(rank_name, api_key):
    if rank_name == 'grandmaster':
        api_link = 'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
    elif rank_name == 'master':
        api_link = 'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
    elif rank_name == 'challenger':
        api_link = 'https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
    r = requests.get(api_link)
    league_df = pd.DataFrame(r.json())

    league_df.reset_index(inplace=True) # 수집한 그마데이터 index정리
    league_entries_df = pd.DataFrame(dict(league_df['entries'])).T # dict구조로 되어 있는 entries컬럼 풀어주기
    league_df = pd.concat([league_df, league_entries_df], axis=1) # 열끼리 결합

    league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)
    league_df.info()
    league_df.to_csv(rank_name+'.csv', index=False, encoding='cp949') # 중간저장
    # league_df.to_csv(rank_name + '.csv', index=False, encoding='utf-8')
    # print(rank_name+"data 받아오기 완료")

# 소환사 고유 닉네임(summonerName)를 이용하여 유저별 고유 accountId 수집


def get_accountid(rank_name, api_key):
    #league_df = pd.read_csv("./" + rank_name + ".csv", encoding='ISO-8859-1')
    league_df = pd.read_csv("./" + rank_name + ".csv", encoding='cp949')
    # league_df = pd.read_csv("./" + rank_name + ".csv", encoding='utf-8')
    # league_df = pd.concat([league_df, account_id_df])
    league_df['accountId'] = 0
    cc = 0
    for i in range(len(league_df)):
        print("고유 accountId 수집시작")
        print(cc)
        cc = cc + 1
        try:
            sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[
                i] + '?api_key=' + api_key
            r = requests.get(sohwan)

            while r.status_code == 429:
                time.sleep(5)
                sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[
                    i] + '?api_key=' + api_key
                r = requests.get(sohwan)

            account_id = r.json()['accountId']
            league_df.iloc[i, -1] = account_id

        except:
            pass

    league_df.info()
    league_df.to_csv(rank_name+'_1.csv', index=False, encoding='cp949')# 중간저장


# request sample
# https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/RuQbIvXeiPbKtBlc3tMbRopspZfrfPZfRykVgky61av0OhUDtze47k3j?season=13
def get_match_data(rank_name, api_key):
    match_info_df = pd.DataFrame()
    league_df3 = pd.read_csv("./"+rank_name+"_1.csv", encoding='cp949')
    season = str(13)
    # print(match_info_df)
    cc = 0
    for i in range(len(league_df3)):
        print("매치데이터 수집 진행중")
        print(cc)
        cc = cc + 1
        try:
            match0 = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + league_df3['accountId'].iloc[
                i] + '?season=' + season + '&api_key=' + api_key
            r = requests.get(match0)

            while r.status_code == 429:
                time.sleep(5)
                match0 = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + league_df3['accountId'].iloc[
                    i] + '?season=' + season + '&api_key=' + api_key   # account_id
                r = requests.get(match0)

            match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()['matches'])])

        except:
            print(i)

    match_info_df.info()
    match_info_df.to_csv(rank_name+"_2.csv", index=False, encoding='cp949')  # 중간저장2

# MatchId(gameId)를 활용한 최종 경기 데이터 수집
# match_info_df2 = match_info_df
# print(match_info_df2)
def get_final_data(rank_name, api_key):
    # match_info_df2 = pd.read_csv("./"+rank_name+"_2.csv", encoding='ISO-8859-1')
    match_fin = pd.DataFrame()
    match_info_df2 = pd.read_csv("./" + rank_name + "_2.csv", encoding='cp949')
    cc = 0
    for i in range(len(match_info_df2)):
        print("MatchId(gameId)를 활용한 최종 경기 데이터 수집중")
        print(cc)
        cc = cc + 1
        api_url = 'https://kr.api.riotgames.com/lol/match/v4/matches/' + str(
            match_info_df2['gameId'].iloc[i]) + '?api_key=' + api_key
        r = requests.get(api_url)

        if r.status_code == 200:  # response가 정상이면 바로 맨 밑으로 이동하여 정상적으로 코드 실행
            pass

        elif r.status_code == 429:
            match_fin.to_csv('rank_name' + '_3.csv', index=False, encoding='cp949')  # 중간저장3
            print('api cost full : infinite loop start')
            print('loop location : ', i)
            start_time = time.time()

            while True:  # 429error가 끝날 때까지 무한 루프
                if r.status_code == 429:

                    print('try 10 second wait time')
                    time.sleep(10)

                    r = requests.get(api_url)
                    print(r.status_code)

                elif r.status_code == 200:  # 다시 response 200이면 loop escape
                    print('total wait time : ', time.time() - start_time)
                    print('recovery api cost')
                    break

        elif r.status_code == 503:  # 잠시 서비스를 이용하지 못하는 에러
            match_fin.to_csv('rank_name' + '_3.csv', index=False, encoding='cp949')  # 중간저장3
            print('service available error')
            start_time = time.time()

            while True:
                if r.status_code == 503 or r.status_code == 429:

                    print('try 10 second wait time')
                    time.sleep(10)

                    r = requests.get(api_url)
                    print(r.status_code)

                elif r.status_code == 200:  # 똑같이 response가 정상이면 loop escape
                    print('total error wait time : ', time.time() - start_time)
                    print('recovery api cost')
                    break
        elif r.status_code == 403:  # api갱신이 필요
            match_fin.to_csv('rank_name' + '_3.csv', index=False, encoding='cp949')  # 중간저장3
            print('you need api renewal')
            print('break')
            break

        # 위의 예외처리 코드를 거쳐서 내려왔을 때 해당 코드가 실행될 수 있도록 작성
        mat = pd.DataFrame(list(r.json().values()), index=list(r.json().keys())).T
        match_fin = pd.concat([match_fin, mat])

    match_fin.to_csv(rank_name+'_3.csv', index=False, encoding='cp949')  # 중간저장3

# match_fin.to_csv('matchdata_fin.csv', index=False, encoding='cp949')  # 중간저장3
# 7
# 챔피언아이디를 제외하고 딕셔너리를 뽑는다
def get_dic():
    a_ls = list(data['teams'])
    # team1
    team1_df = pd.DataFrame()
    for i in range(len(a_ls)):
        try:
            a_ls[i][0].pop('bans', None)
            team1 = pd.DataFrame(list(a_ls[i][0].values()), index=list(a_ls[i][0].keys())).T
            team1_df = team1_df.append(team1)
        except:
            pass

    team1_df.index = range(len(team1_df))

    # team2
    team2_df = pd.DataFrame()
    for i in range(len(a_ls)):
        try:
            a_ls[i][1].pop('bans', None)
            team2 = pd.DataFrame(list(a_ls[i][1].values()), index=list(a_ls[i][1].keys())).T
            team2_df = team2_df.append(team2)
        except:
            pass

    team2_df.index = range(len(team2_df))

    # 컬럼으로 풀어준 team1과 team2와 duration의 데이터를 합쳐준다.
    data_team = pd.concat([team1_df, team2_df, data[['gameDuration']]], axis=1)

